# Machine learning homework5
## Preparation
### Import required librarys

In [1]:
import numpy as np
from libsvm.python import svm
from libsvm.python import svmutil

### Read training and testing data

In [2]:
x_train = np.loadtxt('data/X_train.csv', delimiter=',')
x_test = np.loadtxt('data/X_test.csv', delimiter=',')
y_train = np.loadtxt('data/Y_train.csv', delimiter=',')
y_test = np.loadtxt('data/Y_test.csv', delimiter=',')

### Construct problem with training data

In [ ]:
problem = svmutil.svm_problem(y_train, x_train)